In [1]:
from game import *
import random
from torch import optim

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
device = 'cuda'
#device = 'cpu'


In [3]:
class VAE(nn.Module):
    def __init__(self):
      super(VAE, self).__init__()
      z_dim = 100
      self.conv_en1 = nn.Conv2d(2,128,3,padding=1)
      self.conv_en2 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en3 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en4 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en5 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en6 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en7 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en8 = nn.Conv2d(128,128,3,padding=1)
      self.conv_en9 = nn.Conv2d(128,128,3,padding=1)
    
      
      self.batch_en1 = nn.BatchNorm2d(128)
      self.batch_en2 = nn.BatchNorm2d(128)
      self.batch_en3 = nn.BatchNorm2d(128)
      self.batch_en4 = nn.BatchNorm2d(128)
      self.batch_en5 = nn.BatchNorm2d(128)
      self.batch_en6 = nn.BatchNorm2d(128)
      self.batch_en7 = nn.BatchNorm2d(128)
      self.batch_en8 = nn.BatchNorm2d(128)
      self.batch_en9 = nn.BatchNorm2d(128)
 
      self.dense_enc1 = nn.Linear(128*3*3, 1000)
      self.dense_enc2 = nn.Linear(1000, 1000)
      self.dense_encmean = nn.Linear(1000, z_dim)
      self.dense_encvar = nn.Linear(1000, z_dim)
      
      self.dense_dec1 = nn.Linear(z_dim, 1000)
      self.dense_dec2 = nn.Linear(1000, 128*3*3)
      
      self.conv_dec1 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec2 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec3 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec4 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec5 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec6 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec7 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec8 = nn.Conv2d(128,128,3,padding=1)
      self.conv_dec9 = nn.Conv2d(128,2,3,padding=1)
        
      self.batch_dec1 = nn.BatchNorm2d(128)
      self.batch_dec2 = nn.BatchNorm2d(128)
      self.batch_dec3 = nn.BatchNorm2d(128)
      self.batch_dec4 = nn.BatchNorm2d(128)
      self.batch_dec5 = nn.BatchNorm2d(128)
      self.batch_dec6 = nn.BatchNorm2d(128)
      self.batch_dec7 = nn.BatchNorm2d(128)
      self.batch_dec8 = nn.BatchNorm2d(128)
      self.batch_dec9 = nn.BatchNorm2d(2)

    
    def _encoder(self, x):
      h1 = F.relu(self.batch_en1(self.conv_en1(x)))
    
      h2 = F.relu(self.batch_en2(self.conv_en2(h1)))
      h3 = F.relu(self.batch_en3(self.conv_en3(h2))+h1)
      
      h4 = F.relu(self.batch_en4(self.conv_en4(h3)))
      h5 = F.relu(self.batch_en5(self.conv_en5(h4))+h3)
      
      h6 = F.relu(self.batch_en6(self.conv_en6(h5)))
      h7 = F.relu(self.batch_en7(self.conv_en7(h6))+h5)
    
      h8 = F.relu(self.batch_en8(self.conv_en8(h7)))
      h9 = F.relu(self.batch_en9(self.conv_en9(h8))+h5)
    
        
      x = h9.reshape(-1,128*3*3)
      x = F.relu(self.dense_enc1(x))
      x = F.relu(self.dense_enc2(x))
      mean = self.dense_encmean(x)
      var = F.softplus(self.dense_encvar(x))
      return mean, var
    
    def _sample_z(self, mean, var):
      epsilon = torch.randn(mean.shape).to(device)
      return mean + torch.sqrt(var) * epsilon
 
    def _decoder(self, z):
      x = F.relu(self.dense_dec1(z))
      x = F.relu(self.dense_dec2(x))
      x = x.reshape(-1,128,3,3)
    
      h1 = F.relu(self.batch_dec1(self.conv_dec1(x)))
      h2 = F.relu(self.batch_dec2(self.conv_dec2(h1))+x)
    
      h3 = F.relu(self.batch_dec3(self.conv_dec3(h2)))  
      h4 = F.relu(self.batch_dec4(self.conv_dec4(h3))+h2)
    
      h5 = F.relu(self.batch_dec5(self.conv_dec5(h4)))
      h6 = F.relu(self.batch_dec6(self.conv_dec6(h5))+h4)

      h7 = F.relu(self.batch_dec7(self.conv_dec7(h6)))
      h8 = F.relu(self.batch_dec8(self.conv_dec8(h7))+h7)
    
      h9 = (self.batch_dec9(self.conv_dec9(h8)))
  
      x = self.batch_dec9(h9)
      x = F.sigmoid(x)
      return x

    def forward(self, x):
      mean, var = self._encoder(x)
      z = self._sample_z(mean, var)
      x = self._decoder(z)
      return x, z
    
    def loss(self, x):
      mean, var = self._encoder(x)
      KL = -0.5 * torch.mean(torch.sum(1 + torch.log(var) - mean**2 - var))
      z = self._sample_z(mean, var)
      y = self._decoder(z)
      reconstruction = torch.mean(torch.sum(x * torch.log(y) + (1 - x) * torch.log(1 - y)))
      lower_bound = [-KL, reconstruction]                                      
      return -sum(lower_bound)

In [4]:
state_list = []
for i in range(100):
    state = State()
    while True:
        if state.is_done():
            break
        state = state.next(random_action(state))
        state_list.append(state)

In [5]:
model = VAE()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
BATCH_SIZE = 10
for epoch in range(1000):
    losses = []
    random.shuffle(state_list)
    for index in range(0,len(state_list)-BATCH_SIZE,BATCH_SIZE):
        xs = [[state_list[index+i].pieces,state_list[index+i].enemy_pieces] for i in range(BATCH_SIZE)]
        xs = np.array(xs)
        xs = xs.reshape(-1,2,3,3)
        x = torch.tensor(xs,dtype=torch.float)
        x = x.to(device)
        #print(x)
        model.zero_grad()
        y = model(x)
        loss = model.loss(x) / BATCH_SIZE
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().detach().numpy())
    print("EPOCH: {} loss: {}".format(epoch, np.average(losses)))

/home/tugajin/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


EPOCH: 0 loss: 14.061946868896484
EPOCH: 1 loss: 12.855633735656738
EPOCH: 2 loss: 12.754068374633789
EPOCH: 3 loss: 12.10598087310791
EPOCH: 4 loss: 11.82410717010498
EPOCH: 5 loss: 11.54025650024414
EPOCH: 6 loss: 11.292502403259277
EPOCH: 7 loss: 11.200153350830078
EPOCH: 8 loss: 10.932554244995117
EPOCH: 9 loss: 10.806829452514648
EPOCH: 10 loss: 10.651082992553711
EPOCH: 11 loss: 10.554573059082031
EPOCH: 12 loss: 10.356952667236328
EPOCH: 13 loss: 10.316686630249023
EPOCH: 14 loss: 10.214771270751953
EPOCH: 15 loss: 10.160552978515625
EPOCH: 16 loss: 10.192907333374023
EPOCH: 17 loss: 10.077779769897461
EPOCH: 18 loss: 9.999998092651367
EPOCH: 19 loss: 10.022961616516113
EPOCH: 20 loss: 10.02385425567627
EPOCH: 21 loss: 10.032109260559082
EPOCH: 22 loss: 9.969751358032227
EPOCH: 23 loss: 10.023158073425293
EPOCH: 24 loss: 10.010542869567871
EPOCH: 25 loss: 10.022395133972168
EPOCH: 26 loss: 9.986715316772461
EPOCH: 27 loss: 10.015621185302734
EPOCH: 28 loss: 9.954010009765625
EPO

In [6]:
model.eval()
BATCH_SIZE = 1
for index in range(0,len(state_list)-BATCH_SIZE,BATCH_SIZE):
    print(index)
    xs = [[state_list[index+i].pieces,state_list[index+i].enemy_pieces] for i in range(BATCH_SIZE)]
    xs = np.array(xs)
    xs = xs.reshape(-1,2,3,3)
    x = torch.tensor(xs,dtype=torch.float)
    x = x.to(device)
    print(x)
    #model.no_grad()
    y,z = model(x)
    #print(y)
    #y_item = y.item()
    #print(y_item)
    #print(y_item[1])
    y2 = model._decoder(z).view(-1, 2,3,3)
    #y2_item = y2.item()
    print(y2)
    #print(y2_item)
    #print(y2_item[1])
    

0
tensor([[[[1., 1., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 1.],
          [0., 0., 0.],
          [1., 0., 0.]]]], device='cuda:0')
tensor([[[[0.2187, 0.1822, 0.1489],
          [0.1647, 0.1878, 0.1877],
          [0.1502, 0.1919, 0.1599]],

         [[0.1613, 0.2278, 0.2547],
          [0.2462, 0.2195, 0.2170],
          [0.2587, 0.2077, 0.2504]]]], device='cuda:0', grad_fn=<ViewBackward>)
1
tensor([[[[0., 1., 1.],
          [0., 0., 0.],
          [0., 1., 0.]],

         [[0., 0., 0.],
          [1., 1., 1.],
          [1., 0., 0.]]]], device='cuda:0')
tensor([[[[0.1406, 0.1724, 0.2226],
          [0.1771, 0.1644, 0.1613],
          [0.1475, 0.2289, 0.1562]],

         [[0.2559, 0.2297, 0.1614],
          [0.2293, 0.2312, 0.2423],
          [0.2609, 0.1900, 0.2533]]]], device='cuda:0', grad_fn=<ViewBackward>)
2
tensor([[[[1., 1., 0.],
          [0., 0., 1.],
          [0., 0., 0.]],

         [[0., 0., 1.],
          [1., 0., 0.],
          [0., 1